# Clara Train SDK Bring your Own Component to AutoML (BYOC-AutoML)

By the end of this notebook you would be able to:
1. Write your own handlers for AutoML to display AutoML summary using [MLFlow](https://mlflow.org/)
2. Write your own controller for a random controller
3. Use AutoML admin tool to control running jobs


## Prerequisites
- Familiar with Clara train main concepts. See [Getting Started Notebook](../GettingStarted/GettingStarted.ipynb)
- Familiar with AutoML. See [AutoML Notebook](./AutoML.ipynb)
- Nvidia GPU with 8Gb of memory  
- Have multiple GPUs is preferred   


### Resources
You could watch the free GTC 2020 talks covering Clara Train SDK 
- [S22563](https://developer.nvidia.com/gtc/2020/video/S22563)
Clara train Getting started: cover basics, BYOC, AIAA, AutoML 


## DataSet 
This notebook uses sample dataset (Single image of spleen dataset) provided in the package to train small networks for a couple of epochs. 
This single file is duplicated 32 times in the training set and 9 times to mimic the full spleen data set. 


# Lets get started
Before we get started lets check that we have an NVIDIA GPU available in the docker by running the cell below

In [ ]:
# following command should show all gpus available 
!nvidia-smi

Next cell we define root path for AutoML

In [ ]:
MMAR_ROOT="/claraDevDay/AutoML/"
print ("setting MMAR_ROOT=",MMAR_ROOT)
%ls $MMAR_ROOT


Next cell defines functions that we will use throughout the notebook

In [ ]:
!chmod 777 $MMAR_ROOT/commands/*
def runAutoML(configPath,HANDLER_JSON):
    %cd $MMAR_ROOT/commands
    ! ./automlBYOC.sh $configPath $HANDLER_JSON
def printFile(filePath,lnSt,lnOffset):
    print ("showing ",str(lnOffset)," lines from file ",filePath, "starting at line",str(lnSt))
    lnOffset=lnSt+lnOffset
    !< $filePath head -n "$lnOffset" | tail -n +"$lnSt"


# 1- Writing your own Handler 

## [MLFLOW](https://mlflow.org/)
MLflow is an open source platform to manage the ML lifecycle, 
including experimentation, reproducibility, deployment, and a central model registry. 

Here, we will integrate with MLFlow using BYO Handler to show the results of AutoML experiments. 
We will walk through installing MLFlow, start up the web UI service, then write the BYO Handler.   

### Install MLflow
In this cell below will install MLflow for more in information see MLflow Documentation

In [ ]:
!pip install jinja2==2.11.1
!pip install mlflow


### Start the UI web using 80 which maps to 5000 external  

In [ ]:
import subprocess
%cd /claraDevDay/AutoML
%pwd
a = subprocess.Popen(["mlflow","ui","-h","0.0.0.0","-p","80"])


## Changes to AutoML scripts
AutoML is explained in details in [AutoML notebook](AutoML.ipynb). 
Here, we will use same examples as in AutoML notebook and focus on new BYO handlers
- [automlBYOC.sh](./commands/automlBYOC.sh) is where we configure the number of workers and which gpus to use. 
In this notebook we will use very small networks that will use <2GB. 
We are setting number of workers to 8 and they will all use GPU 0 by specifying 
`workers=0:0:0:0:0:0:0:0`. 
Moreover we set the automlconf to point to a new config which includes our BYO handlers
`--automlconf config_automl_BYO_handler.json`

lets check out the automlBYOC.sh

In [ ]:
configFile=MMAR_ROOT+"/commands/automlBYOC.sh"
printFile(configFile,0,30)

Now lets see the contenet of the new BYO handler we set in the script above 
- [config_automl_BYO_handler.json](./config/config_automl_BYO_handler.json) where any custom controller would be defined 
along with maximum number of MMARs to search and the number of MMARs to keep. </br>
Note we add our handler as 
` "path": "myAutoMLController.MyHandler"`

In [ ]:
configFile=MMAR_ROOT+"/config/config_automl.json"
printFile(configFile,0,30)


- [automl_train_round.sh](./commands/automl_train_round.sh) 
This is the script which will be triggered in each job  


## Write your handler to write to MLFlow 

To write your handler you need to implement `Handler` class found in 
`automl.components.handlers.handler`. You can then implement the following functions:
- `recommendations_available(self, ctx):`
- `startup(self, ctx: Context)`
- `shutdown(self, ctx: Context)`
- `start_job(self, ctx: Context)`
- `round_ended(self, ctx: Context)`
- `end_job(self, ctx: Context)`

In this example we have used the `end_job` function to get the parameters and the final accuracy 
then write it to mlflow. 
Cell below prints out the function we wrote  


Now lets use the [trn_autoML_Enum.json](./config/trn_autoML_Enum.json) to see how it works. 
Recall from basic autoML notebook that this file uses Enum option to search different network architecture arguments for SegResnet network)

lets run it a see configurations generated


In [ ]:
runAutoML("trn_autoML_Enum","config_automl_BYO_handler.json")


Now you can navigate to <yourIP>:5000 and see the MLFlow results shown. 
It should look similar to image below

<br>![mlflow](screenShots/MLFlow.png)  


## 2- Write your own controller     
In this Example, we will write a random controller to use instead of the 
reinforcement learning controller used by AutoML. 


In [ ]:
configFile=MMAR_ROOT+"/BYOC/myAutoMLController.py"
printFile(configFile,27,20)


To write a controller you need to implement the `Controller` class found in `automl.components.controllers.controller`. 
You will need to implement the following functions:
- `set_search_space(self, space, ctx)`
- `initial_recommendation(self, ctx)`
- `refine_recommendation(self, outcome: Outcome, ctx: Context)`

Now lets examine the code to do random pick for enums 

In [ ]:
configFile=MMAR_ROOT+"/config/config_automl_BYO_RandomController.json"
printFile(configFile,27,20)

Lets now run autoML with our controller

In [ ]:
runAutoML("trn_autoML_Loss_Optimizer","config_automl_BYO_RandomController.json")

We can see debugging statements showing our custom code running. 
Finally, you can use tensorboard text tab to see the the parameters passed from the random controller to AutoML.  

# 3- Using the AutoML admin tool 

Once you start AutoML, user can now use use the autoMl admin. 
To get started lets run a simple autoML run using cell below 


In [ ]:
%cd $MMAR_ROOT/commands
! ./automl.sh trn_autoML_Enum 


To start the AutoML admin you can run 
```
python -m dlmed.hci.tools.admin \
--host localhost \
--port 33330 \
--prompt 'AutoML> '
``` 
or simply run [automl_admin.sh](commands/automl_admin.sh) 
This is a shell which will allow you to control automl workers. 
To see list of commands type `?` to see list of commands as below 
<br>![](screenShots/AdminCmds.png)


You can list workers using `list_workers` so you should see some thing similar to 
```
AutoML> list_workers
------------------------------
| NAME | GPUS | STATE | JOBS |
------------------------------
| W1   | 2    | busy  | 3    |
| W2   | 3    | busy  | 3    |
------------------------------
```  



Running `summary` should produce something similar to

```
AutoML summary
Top MMARs to keep: 30
-------------------------------------------------------------------------------------------
| START                      | END                        | ROOT    | SCORE      | WORKER |
-------------------------------------------------------------------------------------------
| 2020-08-13 21:42:46.352932 | 2020-08-13 21:43:13.502620 | W1_1_J1 | -1000000.0 | W1     |
| 2020-08-13 21:42:47.354996 | 2020-08-13 21:43:15.417571 | W2_1_J2 | -1000000.0 | W2     |
-------------------------------------------------------------------------------------------

``` 


You can also manipulate worker as:
- resume_worker
- suspend_worker
 or shut down all AutoML using `stop` command

